In [1]:
import numpy as np

In [2]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

In [3]:
postingList, classVec = loadDataSet()

In [4]:
def createVocabList(dataSet):
    vocabSet = set([])
    for data in dataSet:
        vocabSet = vocabSet | set(data)
    return list(vocabSet)

In [5]:
#词集，在该集合种，每个单词出现就记为1
def setOfWord2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word : %s is not in my vocabulary!"%word)
    return returnVec

#词袋，在该模型种，单词出现一次就+1
def bagOfWord2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print("the word : %s is not in my vocabulary!"%word)
    return returnVec

In [6]:
#词集函数小测试
wordBag = createVocabList(postingList)
vec = setOfWord2Vec(wordBag, ['food', 'cat', 'pig', 'I'])

the word : cat is not in my vocabulary!
the word : pig is not in my vocabulary!


In [7]:
def trainBayes(trainMat, category):
    numTrain = len(trainMat)
    numFeat = len(trainMat[0])
    probA = sum(category)/float(numTrain)
    #防止下溢
#     p0Num = np.zeros(numFeat)
#     p1Num = np.zeros(numFeat)
#     p0Sum = 0.0
#     p1Sum = 0.0
    p0Num = np.ones(numFeat)
    p1Num = np.ones(numFeat)
    p0Sum = 2.0
    p1Sum = 2.0
    for i in range(numTrain):
        if category[i] == 1:
            p1Num += trainMat[i]
            p1Sum += sum(trainMat[i])
        else:
            p0Num += trainMat[i]
            p0Sum += sum(trainMat[i])
    p1Vec = np.log(p1Num / p1Sum)
    p0Vec = np.log(p0Num / p0Sum)
    return probA, p0Vec, p1Vec

In [8]:
#将训练数据转换为向量形式，用0 1表示
trainMat = []
for row in postingList:
    trainMat.append(setOfWord2Vec(wordBag, row))

In [9]:
pA, p0, p1 = trainBayes(trainMat, classVec)

In [10]:
#predict
def predict(wordVec, p0Vec, p1Vec, pA):
    #侮辱
    p1 = sum(p1Vec * wordVec) + np.log(pA)
    #非侮辱
    p0 = sum(p0Vec * wordVec) + np.log(1-pA)
    if p1 > p0:
        return 1;
    else:
        return 0;

In [11]:
testWord = ['I', 'love', 'you']
testVec = np.array(setOfWord2Vec(wordBag, testWord))
print("the prediction is : %d"%predict(testVec, p0, p1, pA))
testWord = ['dog', 'stupid', 'posting']
testVec = np.array(setOfWord2Vec(wordBag, testWord))
print("the prediction is : %d"%predict(testVec, p0, p1, pA))

the word : you is not in my vocabulary!
the prediction is : 0
the prediction is : 1


1.将所有词放入一个集合中
2.将每个词向量转换为0 1表示的向量（在1中出现的词就为1，未出现即未0），放入一个矩阵中，构成训练数据。
3.训练：计算p(c)的概率，统计分类后，每个单词在各自的分类中出现的次数a（a为一个数组），统计在该分类下出现的所有的单词数量b，计算a/b，得到每个单词出现的概率p。
4.计算两个分类下的p * p(c),得到后验概率。输出较大的值。